In [7]:
import xgboost as xgb
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
color_pal = sns.color_palette()
plt.style.use('fivethirtyeight')

In [8]:
df = pd.read_csv(r"..\new_datasets\datos_gramineas.csv")
df = df.set_index("fecha")
df.index = pd.to_datetime(df.index)
df = df.sort_index(ascending=True)

In [9]:
model = xgb.Booster()
model.load_model('modelo_t+1.json')

features_que_pide_el_modelo = model.feature_names

In [10]:
ultima_fila = df.tail(6)

input_data = ultima_fila[features_que_pide_el_modelo]

In [11]:
# Convertir los datos al formato que entiende el Booster
dmatrix_input = xgb.DMatrix(input_data)

# Realizar la predicción
prediccion_raw = model.predict(dmatrix_input)[0]

# Si en tu entrenamiento usaste logaritmo (como en train_model_v4.py), 
# debes revertir la transformación con expm1
prediccion_final = np.expm1(prediccion_raw)

print(f"La predicción de polen para el próximo día es: {prediccion_final:.2f}")

La predicción de polen para el próximo día es: inf


C:\Users\sergi\AppData\Local\Temp\ipykernel_6352\3425150665.py:9: RuntimeWarning: overflow encountered in expm1
  prediccion_final = np.expm1(prediccion_raw)
